In [1]:
import pandas as pd
import pyarrow.parquet as pq
import catboost as cb
import numpy as np
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import json

In [148]:
df = pd.read_parquet('train/ids_target.parquet.fastparquet', engine='pyarrow').set_index('id')

In [149]:
df.head()

,timestamp,client_id,seller_id,target
id,,,,
f90c6b1c-37fa-4e0d-bdb9-bd332180f7cf,2020-06-22 00:00:00,7390515e-26ff-4ece-9276-3aeffd159150,241c7f98-838c-461e-84f6-653f8ee380d7,0
8bd5070f-541a-4acf-99b7-aad83db4ffa9,2023-05-02 20:00:00,8c2ad13a-90bd-414a-86bf-73c97e76e91c,c7aa6d84-ffa9-4122-b662-cfe09d48119e,0
0f57961f-677c-4fb6-95bb-1de6903476b7,2022-11-08 16:00:00,2f5d044d-5d43-4ad4-a53b-2d1e100bfc52,3eeb1798-423b-4306-8017-b49c31a1164d,0
16b48e53-7a87-4916-800e-4c5ac5500262,2022-10-06 15:00:00,cce566a9-1f83-4804-a7d9-94fe17618702,3eeb1798-423b-4306-8017-b49c31a1164d,0
df06c1d2-3601-4d8b-8209-1e3dbb4d80e3,2023-06-21 19:00:00,11d777ac-58cc-4059-8d53-e8fc9287d143,668adc51-6344-476d-a8c3-568ff0efcf57,0


In [150]:
feature_file_names = [
    'train/prelead_features_client_gender.parquet.fastparquet',
    'train/prelead_features_client_history.parquet.fastparquet',
    'train/prelead_features_contractors_banks.parquet.fastparquet',
    'train/prelead_features_contractors_directional.parquet.fastparquet',
    'train/prelead_features_egrul.parquet.fastparquet',
    'train/prelead_features_goszakupki.parquet.fastparquet',
    'train/prelead_features_holding.parquet.fastparquet',
    'train/prelead_features_revenue.parquet.fastparquet',
    'train/prelead_features_time_features.parquet.fastparquet',
    'train/prelead_features_user_info.parquet.fastparquet',
]

In [151]:
for file in feature_file_names:
    data = pd.read_parquet(file, engine='pyarrow').set_index('id')
    df = df.join(data)

In [152]:
df.head()

,timestamp,client_id,seller_id,target,client_gender,days_since_last_take,wrong_phone_last_time,has_new_phones,cnt_takes,cnt_not_sell,...,time_tz_diff,role,division,position,grade,is_teamlead,teamid,hypothesisid,employed_days,seller_gender
id,,,,,,,,,,,,,,,,,,,,,
f90c6b1c-37fa-4e0d-bdb9-bd332180f7cf,2020-06-22 00:00:00,7390515e-26ff-4ece-9276-3aeffd159150,241c7f98-838c-461e-84f6-653f8ee380d7,0,unknown,9999,False,True,1,0,...,0.0,role_5,division_12,position_5,grade_12,False,1f16db72-ccb7-4db5-abd6-a8c0d14239eb,58213a29-eae0-48cd-94f6-851c79b9d813,780.0,женщина
8bd5070f-541a-4acf-99b7-aad83db4ffa9,2023-05-02 20:00:00,8c2ad13a-90bd-414a-86bf-73c97e76e91c,c7aa6d84-ffa9-4122-b662-cfe09d48119e,0,unknown,9999,False,True,1,0,...,0.0,role_9,division_1,position_51,grade_0,False,57d5e17c-80ce-47cd-921b-ced9fdae0655,0790e3e3-d4f3-4b5a-9e1f-925c70d88b84,321.0,мужчина
0f57961f-677c-4fb6-95bb-1de6903476b7,2022-11-08 16:00:00,2f5d044d-5d43-4ad4-a53b-2d1e100bfc52,3eeb1798-423b-4306-8017-b49c31a1164d,0,unknown,9999,False,True,1,0,...,2.0,role_9,division_8,unknown,grade_18,False,f0a7636d-a0f4-422e-94bc-a1839d8c3b02,0790e3e3-d4f3-4b5a-9e1f-925c70d88b84,-1.0,мужчина
16b48e53-7a87-4916-800e-4c5ac5500262,2022-10-06 15:00:00,cce566a9-1f83-4804-a7d9-94fe17618702,3eeb1798-423b-4306-8017-b49c31a1164d,0,unknown,9999,False,True,1,0,...,-2.0,role_9,division_8,unknown,grade_18,False,f0a7636d-a0f4-422e-94bc-a1839d8c3b02,0790e3e3-d4f3-4b5a-9e1f-925c70d88b84,-1.0,мужчина
df06c1d2-3601-4d8b-8209-1e3dbb4d80e3,2023-06-21 19:00:00,11d777ac-58cc-4059-8d53-e8fc9287d143,668adc51-6344-476d-a8c3-568ff0efcf57,0,женщина,9999,False,True,1,0,...,NaN,role_3,division_8,unknown,unknown,False,95f82126-a39f-4c16-b452-998c43709af6,5aa4da89-7f72-49c3-bb8d-39b63b24e2a2,-1.0,женщина


In [153]:
df = df.drop([
    'timestamp',
    'client_id',
    'seller_id',
    'teamid',
    'hypothesisid'
], axis=1)

In [154]:
df.head()

,target,client_gender,days_since_last_take,wrong_phone_last_time,has_new_phones,cnt_takes,cnt_not_sell,tochka_contractor_bank_top_1,tochka_contractors_n_incoming,tochka_contractors_count_sum_rub_incoming,...,holding_avg_days_since_take,revenue,time_tz_diff,role,division,position,grade,is_teamlead,employed_days,seller_gender
id,,,,,,,,,,,,,,,,,,,,,
f90c6b1c-37fa-4e0d-bdb9-bd332180f7cf,0,unknown,9999,False,True,1,0,unknown,0,0,...,unknown,unknown,0.0,role_5,division_12,position_5,grade_12,False,780.0,женщина
8bd5070f-541a-4acf-99b7-aad83db4ffa9,0,unknown,9999,False,True,1,0,unknown,0,0,...,unknown,unknown,0.0,role_9,division_1,position_51,grade_0,False,321.0,мужчина
0f57961f-677c-4fb6-95bb-1de6903476b7,0,unknown,9999,False,True,1,0,unknown,0,0,...,unknown,unknown,2.0,role_9,division_8,unknown,grade_18,False,-1.0,мужчина
16b48e53-7a87-4916-800e-4c5ac5500262,0,unknown,9999,False,True,1,0,unknown,0,0,...,unknown,unknown,-2.0,role_9,division_8,unknown,grade_18,False,-1.0,мужчина
df06c1d2-3601-4d8b-8209-1e3dbb4d80e3,0,женщина,9999,False,True,1,0,bank_388,0,0,...,unknown,<10000000,NaN,role_3,division_8,unknown,unknown,False,-1.0,женщина


In [155]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.3, random_state=42, stratify=df['target'])

In [156]:
# Удаление столбцов с одинаковыми значениями во всех строках
X_train = X_train.loc[:, X_train.nunique() > 1]
X_test = X_test.loc[:, X_test.nunique() > 1]

In [157]:
# Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['int64', 'float']))
X_test_scaled = scaler.transform(X_test.select_dtypes(include=['int64', 'float']))

In [158]:
# Получение списка столбцов для нормализованных признаков
scaled_columns = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Объединение нормализованных признаков
X_train = pd.DataFrame(X_train_scaled, columns=scaled_columns)
X_test = pd.DataFrame(X_test_scaled, columns=scaled_columns)

In [111]:
# Оптимизация гиперпараметров модели CatBoost
space = {
    'iterations': hp.choice('iterations', range(300, 700)),
    'learning_rate': hp.choice('learning_rate', np.logspace(-3, -1, 10)),
    'depth': hp.choice('depth', range(3, 10)),
    'l2_leaf_reg': hp.choice('l2_leaf_reg', np.logspace(-3, 3, 10)),
    'border_count': hp.choice('border_count', range(10, 256)),
    'random_state': 42
}

In [112]:
def objective(params):
    model = cb.CatBoostClassifier(**params)
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    y_pred = cross_val_predict(model, X_train, y_train, cv=kfold, method='predict_proba')[:, 1]
    score = f1_score(y_train, np.where(y_pred >= 0.5, 1, 0))
    return {'loss': -score, 'status': 'ok'}

In [113]:
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=30, trials=trials)

0:	learn: 0.6895401	total: 52.8ms	remaining: 31.5s    

1:	learn: 0.6858208	total: 112ms	remaining: 33.5s     

2:	learn: 0.6823765	total: 174ms	remaining: 34.5s     

3:	learn: 0.6789336	total: 238ms	remaining: 35.3s     

4:	learn: 0.6752507	total: 307ms	remaining: 36.4s     

5:	learn: 0.6716970	total: 381ms	remaining: 37.5s     

6:	learn: 0.6683742	total: 457ms	remaining: 38.5s     

7:	learn: 0.6647789	total: 530ms	remaining: 39s       

8:	learn: 0.6613555	total: 594ms	remaining: 38.8s     

9:	learn: 0.6580146	total: 652ms	remaining: 38.2s     

10:	learn: 0.6544537	total: 716ms	remaining: 38.1s    

11:	learn: 0.6512040	total: 782ms	remaining: 38.1s    

12:	learn: 0.6476689	total: 846ms	remaining: 38s      

13:	learn: 0.6444243	total: 924ms	remaining: 38.5s    

14:	learn: 0.6409876	total: 1s	remaining: 38.8s       

15:	learn: 0.6377671	total: 1.08s	remaining: 39.2s    

16:	learn: 0.6343775	total: 1.15s	remaining: 39.3s    

17:	learn: 0.6312223	total: 1.21s	remaining: 39.

KeyboardInterrupt: 

In [31]:
# Преобразуем значения типа numpy.int64 в обычные целые числа
best_dict = {k: int(v) if isinstance(v, np.int64) else v for k, v in best.items()}

with open('best_params.json', 'w') as f:
    json.dump(best_dict, f)

In [159]:
categorical_features = X_train.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

In [160]:
with open('best_params.json', 'r') as f:
    best_params = json.load(f)

In [161]:
model = cb.CatBoostClassifier(**best_params, auto_class_weights='Balanced')
model.fit(X_train, y_train, cat_features=categorical_features, early_stopping_rounds=50)

0:	learn: 0.6926310	total: 88.5ms	remaining: 44.2s
1:	learn: 0.6921696	total: 169ms	remaining: 42s
2:	learn: 0.6916825	total: 266ms	remaining: 44s
3:	learn: 0.6911317	total: 377ms	remaining: 46.8s
4:	learn: 0.6906369	total: 506ms	remaining: 50.1s
5:	learn: 0.6902092	total: 688ms	remaining: 56.6s
6:	learn: 0.6896706	total: 784ms	remaining: 55.2s
7:	learn: 0.6892258	total: 862ms	remaining: 53s
8:	learn: 0.6887571	total: 947ms	remaining: 51.7s
9:	learn: 0.6882957	total: 1.04s	remaining: 51.2s
10:	learn: 0.6878381	total: 1.14s	remaining: 50.7s
11:	learn: 0.6874766	total: 1.23s	remaining: 49.8s
12:	learn: 0.6869973	total: 1.32s	remaining: 49.4s
13:	learn: 0.6865123	total: 1.41s	remaining: 48.9s
14:	learn: 0.6861960	total: 1.5s	remaining: 48.7s
15:	learn: 0.6858444	total: 1.61s	remaining: 48.7s
16:	learn: 0.6854140	total: 1.73s	remaining: 49.2s
17:	learn: 0.6850768	total: 1.82s	remaining: 48.9s
18:	learn: 0.6846653	total: 1.9s	remaining: 48.2s
19:	learn: 0.6842787	total: 1.98s	remaining: 47.

In [162]:
models = [
    cb.CatBoostClassifier(**best_params, auto_class_weights='Balanced'),
    cb.CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, random_state=42, auto_class_weights='Balanced'),
    cb.CatBoostClassifier(iterations=500, learning_rate=0.01, depth=8, random_state=42, auto_class_weights='Balanced')
]

X_train_stacked = pd.DataFrame()
X_test_stacked = pd.DataFrame()

In [163]:
for i, model in enumerate(models):
    model.fit(X_train, y_train, cat_features=categorical_features, early_stopping_rounds=50)
    y_pred_train = model.predict_proba(X_train)[:, 1]
    y_pred_test = model.predict_proba(X_test)[:, 1]
    y_pred_train_series = pd.Series(y_pred_train, name=f'model_{i}')
    y_pred_test_series = pd.Series(y_pred_test, name=f'model_{i}')
    X_train_stacked = pd.concat([X_train_stacked, y_pred_train_series], axis=1)
    X_test_stacked = pd.concat([X_test_stacked, y_pred_test_series], axis=1)

0:	learn: 0.6926310	total: 78.6ms	remaining: 39.2s
1:	learn: 0.6921696	total: 156ms	remaining: 38.8s
2:	learn: 0.6916825	total: 221ms	remaining: 36.6s
3:	learn: 0.6911317	total: 306ms	remaining: 38s
4:	learn: 0.6906369	total: 407ms	remaining: 40.3s
5:	learn: 0.6902092	total: 496ms	remaining: 40.8s
6:	learn: 0.6896706	total: 602ms	remaining: 42.4s
7:	learn: 0.6892258	total: 714ms	remaining: 43.9s
8:	learn: 0.6887571	total: 822ms	remaining: 44.8s
9:	learn: 0.6882957	total: 922ms	remaining: 45.2s
10:	learn: 0.6878381	total: 998ms	remaining: 44.4s
11:	learn: 0.6874766	total: 1.07s	remaining: 43.6s
12:	learn: 0.6869973	total: 1.2s	remaining: 45s
13:	learn: 0.6865123	total: 1.32s	remaining: 45.8s
14:	learn: 0.6861960	total: 1.41s	remaining: 45.5s
15:	learn: 0.6858444	total: 1.49s	remaining: 45s
16:	learn: 0.6854140	total: 1.57s	remaining: 44.7s
17:	learn: 0.6850768	total: 1.66s	remaining: 44.4s
18:	learn: 0.6846653	total: 1.75s	remaining: 44.4s
19:	learn: 0.6842787	total: 1.85s	remaining: 44

In [164]:
meta_model = cb.CatBoostClassifier(**best_params, auto_class_weights='Balanced')
meta_model.fit(X_train_stacked, y_train)

0:	learn: 0.6909276	total: 89.9ms	remaining: 44.9s
1:	learn: 0.6890524	total: 172ms	remaining: 42.8s
2:	learn: 0.6870715	total: 268ms	remaining: 44.4s
3:	learn: 0.6853081	total: 390ms	remaining: 48.4s
4:	learn: 0.6835542	total: 481ms	remaining: 47.6s
5:	learn: 0.6817059	total: 580ms	remaining: 47.8s
6:	learn: 0.6800219	total: 692ms	remaining: 48.7s
7:	learn: 0.6781225	total: 786ms	remaining: 48.4s
8:	learn: 0.6762011	total: 876ms	remaining: 47.8s
9:	learn: 0.6744714	total: 966ms	remaining: 47.3s
10:	learn: 0.6728574	total: 1.05s	remaining: 46.9s
11:	learn: 0.6710947	total: 1.14s	remaining: 46.5s
12:	learn: 0.6693248	total: 1.23s	remaining: 46.1s
13:	learn: 0.6675285	total: 1.31s	remaining: 45.5s
14:	learn: 0.6659489	total: 1.39s	remaining: 44.9s
15:	learn: 0.6645347	total: 1.47s	remaining: 44.4s
16:	learn: 0.6629428	total: 1.55s	remaining: 43.9s
17:	learn: 0.6615097	total: 1.62s	remaining: 43.4s
18:	learn: 0.6600423	total: 1.7s	remaining: 43.1s
19:	learn: 0.6585950	total: 1.78s	remaini

In [165]:
# Evaluate the meta model
y_pred = meta_model.predict(X_test_stacked)
f1 = f1_score(y_test, y_pred)
print('F1-score:', f1)

F1-score: 0.031315856588679046


In [166]:
# Load the test data and set the index
test_df = pd.read_parquet('test/ids_target.parquet.fastparquet', engine='pyarrow').set_index('id')

In [167]:
test_df.head()

,timestamp,client_id,seller_id
id,,,
21ea07a7-4c9c-4b9a-b0ae-921e7e562312,2023-09-08 15:00:00,f9939775-f1ab-4bb5-b65f-f14e0db1dbe1,91d15841-374c-4332-b0e1-86761eebb206
63f622c9-e4f6-4ec1-bb92-c551296ea8e9,2024-01-29 17:00:00,d649a6c9-aa3c-452c-9037-52ed1c474dc7,e2fc3cfa-7c61-4498-9705-844cc9fee32c
c26e5b5e-f56d-48aa-89dc-711af7f9510e,2024-02-27 11:00:00,2665e9b2-99a3-4d5c-9425-41c0eba56d25,b23b8476-0eec-403e-905a-1f984a9d7802
7d7cc2bf-e392-4f6b-af76-b2cef6a2d5b2,2023-09-25 18:00:00,172935cd-1887-4f8a-8826-a2d7b350e1e2,41d7a1a5-1ade-4eab-8c4a-b9a93255a5fd
e638d6ed-4522-4e39-aedf-3db5ea68d185,2024-02-20 05:00:00,1de0ffd5-86f4-4bf7-be67-5e3f82d7da20,573cb578-9b95-4eba-95ad-09b7a198be55


In [168]:
test_feature_file_names = [
    'test/prelead_features_client_gender.parquet.fastparquet',
    'test/prelead_features_client_history.parquet.fastparquet',
    'test/prelead_features_contractors_banks.parquet.fastparquet',
    'test/prelead_features_contractors_directional.parquet.fastparquet',
    'test/prelead_features_egrul.parquet.fastparquet',
    'test/prelead_features_goszakupki.parquet.fastparquet',
    'test/prelead_features_holding.parquet.fastparquet',
    'test/prelead_features_revenue.parquet.fastparquet',
    'test/prelead_features_time_features.parquet.fastparquet',
    'test/prelead_features_user_info.parquet.fastparquet',
]

In [169]:
for file in test_feature_file_names:
    data = pd.read_parquet(file, engine='pyarrow').set_index('id')
    test_df = test_df.join(data)

In [170]:
test_df = test_df.drop([
    'timestamp',
    'client_id',
    'seller_id',
    'teamid',
    'hypothesisid'
], axis=1)

In [171]:
test_df.head()

,client_gender,days_since_last_take,wrong_phone_last_time,has_new_phones,cnt_takes,cnt_not_sell,tochka_contractor_bank_top_1,tochka_contractors_n_incoming,tochka_contractors_count_sum_rub_incoming,tochka_contractors_avg_sum_rub_incoming,...,holding_avg_days_since_take,revenue,time_tz_diff,role,division,position,grade,is_teamlead,employed_days,seller_gender
id,,,,,,,,,,,,,,,,,,,,,
21ea07a7-4c9c-4b9a-b0ae-921e7e562312,unknown,9999,False,True,1,0,bank_265,0,0,< 10000,...,unknown,unknown,NaN,role_5,division_0,position_22,grade_0,False,231.0,женщина
63f622c9-e4f6-4ec1-bb92-c551296ea8e9,unknown,14,False,False,1,0,unknown,0,0,< 10000,...,unknown,unknown,0.0,role_5,division_12,position_22,unknown,False,543.0,женщина
c26e5b5e-f56d-48aa-89dc-711af7f9510e,unknown,106,False,False,1,0,bank_367,1 - 5,1 - 5,10000 - 100000,...,unknown,unknown,2.0,role_5,division_2,position_22,unknown,False,21.0,женщина
7d7cc2bf-e392-4f6b-af76-b2cef6a2d5b2,мужчина,9999,False,True,1,0,unknown,0,0,< 10000,...,unknown,unknown,-1.0,role_3,division_8,position_22,unknown,False,-24.0,женщина
e638d6ed-4522-4e39-aedf-3db5ea68d185,unknown,9999,False,True,1,0,unknown,0,0,< 10000,...,unknown,unknown,6.0,role_9,division_1,position_51,grade_0,False,613.0,женщина


In [172]:
# Remove columns with unique values
test_df = test_df.loc[:, test_df.nunique() > 1]

# Normalize the data
test_df_scaled = scaler.transform(test_df.select_dtypes(include=['int64', 'float']))

# Create a DataFrame with the scaled data using the training data column order
test_df = pd.DataFrame(test_df_scaled, columns=scaled_columns)

In [173]:
test_df.head()

,days_since_last_take,holding_cnt_takes,holding_cnt_not_sell,time_tz_diff,employed_days
0,0.879972,-0.070487,-0.085279,NaN,-0.329293
1,-1.157455,-0.070487,-0.085279,-0.078951,0.315700
2,-1.138683,-0.070487,-0.085279,0.898342,-0.763423
3,0.879972,-0.006716,-0.085279,-0.567597,-0.856450
4,0.879972,-0.070487,-0.085279,2.852926,0.460410


In [176]:
# Stack the predictions of the models for the test data
test_df_stacked = pd.DataFrame()
for i, model in enumerate(models):
    y_pred_test = model.predict_proba(test_df)[:, 1]
    y_pred_test_series = pd.Series(y_pred_test, name=f'model_{i}')
    test_df_stacked = pd.concat([test_df_stacked, y_pred_test_series], axis=1)

In [177]:
# Do predictions for test data with the meta model
test_predictions = meta_model.predict(test_df_stacked)

In [180]:
submission_df = pd.DataFrame({
    'id': test_df.index,
    'target': test_predictions
})

submission_df.to_csv('submission.csv', index=False)